# Test New Dataset

This serves only for early protyping of new dataset functionalities

In [3]:
from jaxtyping import Int
from pathlib import Path
import pandas as pd

In [4]:
splits = {'train': 'data/train-00000-of-00001-1028f23e353fbe3e.parquet', 'validation': 'data/validation-00000-of-00001-6c7328ff6c84284c.parquet', 'test': 'data/test-00000-of-00001-f0e719df791966ff.parquet'}
df_train = pd.read_parquet("hf://datasets/derek-thomas/ScienceQA/" + splits["train"])
df_validation = pd.read_parquet("hf://datasets/derek-thomas/ScienceQA/" + splits["validation"])
df_test = pd.read_parquet("hf://datasets/derek-thomas/ScienceQA/" + splits["test"])

In [5]:
df_test.columns

Index(['image', 'question', 'choices', 'answer', 'hint', 'task', 'grade',
       'subject', 'topic', 'category', 'skill', 'lecture', 'solution'],
      dtype='object')

In [43]:
def generate_question(df_row: pd.Series
                      ) -> str:
    choices_keys = ["A", "B", "C", "D"]
    question = df_row["question"]
    choices = df_row["choices"]
    answer = df_row["answer"]
    full_question = f"{question}\n"
    for key, choice in zip(choices_keys, choices):
        full_question += f"{key}. {choice}\n"
    full_question += f"Answer:{choices[answer]}"
    return full_question

def generate_shots_per_subject(df: pd.DataFrame,
                               num_shot:Int
                               ) -> pd.DataFrame:
    df = df[df["image"].isnull()]
    shot_df = df.groupby('subject').apply(lambda x: x.sample(num_shot, replace=True)).reset_index(drop=True)
    shot_df['prompt'] = shot_df.apply(generate_question, axis=1)
    return shot_df

def generate_dataset(df: pd.DataFrame, 
                     path_save: Path,
                     num_shot: Int = None):
    pass
prova = generate_shots_per_subject(df_validation, 5)

/tmp/ipykernel_1702156/230055257.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  shot_df = df.groupby('subject').apply(lambda x: x.sample(num_shot, replace=True)).reset_index(drop=True)


In [14]:
df = pd.concat([df_train, df_validation, df_test])
filtered_df = df[df["image"].isnull()]
filtered_df = df[df['choices'].apply(lambda x: len(x) in [3,4])]

In [17]:
filtered_df.category.value_counts()

category
State capitals           1475
Units and measurement     413
Materials                 352
Classification            334
Geography                 307
                         ... 
Earth's features            1
Early China                 1
The Constitution            1
Topographic maps            1
Electricity                 1
Name: count, Length: 93, dtype: int64

In [18]:
counts = filtered_df.category.value_counts()
# save as csv
counts.to_csv("category_counts.csv")
